In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv("../input/sentiment-analysis-on-movie-reviews/train.tsv.zip", sep='\t')
train

In [ ]:
test = pd.read_csv("../input/sentiment-analysis-on-movie-reviews/test.tsv.zip", sep='\t')
test

In [ ]:
sampleSubmission = pd.read_csv("../input/sentiment-analysis-on-movie-reviews/sampleSubmission.csv")
sampleSubmission

In [ ]:
X = train.pop("Phrase")
y = train.pop("Sentiment")

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))

In [ ]:
train_data_batch, train_label_batch = next(iter(dataset.batch(10)))
train_data_batch

In [ ]:
train_label_batch

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
model = keras.Sequential([
    hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(5)
])
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(dataset.shuffle(10000).batch(512),
          epochs=20,
          verbose=1)

In [ ]:
test_x = test.pop("Phrase")
test_x

In [ ]:
test_x = tf.constant(test_x, dtype=tf.string)

In [ ]:
prob_model = keras.Sequential([
    model,
    keras.layers.Softmax()
])

In [ ]:
preds = prob_model.predict(test_x)

In [ ]:
preds